In [3]:
import os
import requests
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications import VGG16, MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from IPython.display import display, Markdown
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, MaxPooling2D
import time
import h5py
import json
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import shutil
import Augmentor
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
batch_size = 30
image_size = (128, 128)
seed = 81

aug_train_dir = r'C:\Users\kevin\OneDrive\Desktop\Thesisrepo\CleanandAugData\train_ds\output'
aug_train_ds = tf.keras.utils.image_dataset_from_directory(
    aug_train_dir,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=seed
)



val_dir = r'C:\Users\kevin\OneDrive\Desktop\Thesisrepo\CleanandAugData\val_ds'
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=seed
)


Found 10000 files belonging to 4 classes.
Found 80 files belonging to 4 classes.


In [ ]:
# VGG16 Model
from tensorflow.keras.optimizers import Adam
vgg_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))  

for layer in vgg_base_model.layers:
    layer.trainable = False

vgg_x = vgg_base_model.output
vgg_x = layers.Flatten()(vgg_x)
vgg_x = layers.Dropout(0.5)(vgg_x)
vgg_predictions = layers.Dense(4, activation='softmax')(vgg_x)

vgg_model = Model(inputs=vgg_base_model.input, outputs=vgg_predictions)

vgg_model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
checkpoint = ModelCheckpoint('vgg_best_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)

vgg_history = vgg_model.fit(aug_train_ds, validation_data=val_ds, epochs=10, callbacks=[checkpoint])

# Generate a timestamp
timestamp = time.strftime("%Y%m%d-%H%M%S")

# Make file name with timestamp
model_name = "vgg_model"
file_path = f"{model_name}_{timestamp}.h5"

# Save the model
vgg_model.save(file_path)
print(f'Model saved as {file_path}')

# Append description and history to .h5 file
description = "Final VGG16 Model"
history_dict = {
    'loss': vgg_history.history['loss'],
    'accuracy': vgg_history.history['accuracy'],
    'val_loss': vgg_history.history['val_loss'],
    'val_accuracy': vgg_history.history['val_accuracy']
}
history_json = json.dumps(history_dict)

with h5py.File(file_path, 'a') as f:
    f.attrs['description'] = description
    f.attrs['training_history'] = history_json

print('Description and training history saved to the model file.')

# Load the description and training history
with h5py.File(file_path, 'r') as f:
    loaded_description = f.attrs['description']
    loaded_history_json = f.attrs['training_history']
    loaded_history = json.loads(loaded_history_json)

print(f'Loaded description: {loaded_description}')
print(f'Loaded history: {loaded_history}')

# Load the saved model
saved_model = load_model(file_path)
saved_model.summary()


Epoch 1/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 220s 651ms/step - accuracy: 0.2420 - loss: 23.4521 - val_accuracy: 0.3500 - val_loss: 14.0577
Epoch 2/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 214s 640ms/step - accuracy: 0.4013 - loss: 14.3497 - val_accuracy: 0.4500 - val_loss: 10.3509
Epoch 3/10
219/334 ━━━━━━━━━━━━━━━━━━━━ 1:17 670ms/step - accuracy: 0.5035 - loss: 10.9069

In [11]:

#ResNet50 Model
resnet_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in resnet_base_model.layers:
    layer.trainable = False


resnet_x = resnet_base_model.output
resnet_x = layers.Flatten()(resnet_x)
resnet_x = layers.Dropout(0.5)(resnet_x)
resnet_predictions = layers.Dense(4, activation='softmax')(resnet_x)

resnet_model = Model(inputs=resnet_base_model.input, outputs=resnet_predictions)

resnet_model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('resnet_best_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)

resnet_history = resnet_model.fit(aug_train_ds, validation_data=val_ds, epochs=10, callbacks=[checkpoint])


timestamp = time.strftime("%Y%m%d-%H%M%S")

# Make file name with timestamp
model_name = "resnet_model"
file_path = f"{model_name}_{timestamp}.h5"

# Save the model
resnet_model.save(file_path)
print(f'Model saved as {file_path}')

# Append description and history to .h5 file
description = "Final ResNet50 Model"
history_dict = {
    'loss': resnet_history.history['loss'],
    'accuracy': resnet_history.history['accuracy'],
    'val_loss': resnet_history.history['val_loss'],
    'val_accuracy': resnet_history.history['val_accuracy']
}
history_json = json.dumps(history_dict)

with h5py.File(file_path, 'a') as f:
    f.attrs['description'] = description
    f.attrs['training_history'] = history_json

print('Description and training history saved to the model file.')

# Load the description and training history
with h5py.File(file_path, 'r') as f:
    loaded_description = f.attrs['description']
    loaded_history_json = f.attrs['training_history']
    loaded_history = json.loads(loaded_history_json)

print(f'Loaded description: {loaded_description}')
print(f'Loaded history: {loaded_history}')

# Load the saved model
saved_model = load_model(file_path)
saved_model.summary()



Epoch 1/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 77s 212ms/step - accuracy: 0.8315 - loss: 8.1285 - val_accuracy: 0.8250 - val_loss: 23.7953
Epoch 2/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 70s 209ms/step - accuracy: 0.9614 - loss: 2.8366 - val_accuracy: 0.8375 - val_loss: 23.3078
Epoch 3/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 69s 205ms/step - accuracy: 0.9725 - loss: 2.1688 - val_accuracy: 0.8250 - val_loss: 22.9390
Epoch 4/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 68s 205ms/step - accuracy: 0.9778 - loss: 1.7669 - val_accuracy: 0.7875 - val_loss: 36.2366
Epoch 5/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 71s 211ms/step - accuracy: 0.9776 - loss: 2.0315 - val_accuracy: 0.8125 - val_loss: 37.1933
Epoch 6/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 72s 216ms/step - accuracy: 0.9841 - loss: 1.4904 - val_accuracy: 0.8125 - val_loss: 43.9535
Epoch 7/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 73s 218ms/step - accuracy: 0.9846 - loss: 1.8209 - val_accuracy: 0.8500 - val_loss: 58.5134
Epoch 8/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 72s 215ms/step - accuracy: 0.9852 - 

Model saved as resnet_model_20241117-205934.h5
Description and training history saved to the model file.
Loaded description: Final ResNet50 Model
Loaded history: {'loss': [6.494207859039307, 3.1136231422424316, 2.1709437370300293, 1.7345281839370728, 1.844712734222412, 1.9207935333251953, 1.7747724056243896, 1.4141329526901245, 1.7063324451446533, 1.7497060298919678], 'accuracy': [0.891700029373169, 0.9603000283241272, 0.9736999869346619, 0.9789999723434448, 0.978600025177002, 0.9810000061988831, 0.9855999946594238, 0.9860000014305115, 0.9868999719619751, 0.9872000217437744], 'val_loss': [23.795318603515625, 23.30782127380371, 22.93899154663086, 36.236602783203125, 37.193328857421875, 43.95348358154297, 58.5134162902832, 43.09272384643555, 53.81024932861328, 76.8603515625], 'val_accuracy': [0.824999988079071, 0.8374999761581421, 0.824999988079071, 0.7875000238418579, 0.8125, 0.8125, 0.8500000238418579, 0.8500000238418579, 0.8374999761581421, 0.8500000238418579]}


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 134, 134, 3)       │               0 │ input_layer_7[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 64, 64, 64)        │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 64, 64, 64)        │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 64, 64, 64)        │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 66, 66, 64)        │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 32, 32, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 32, 32, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 32, 32, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 32, 32, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 32, 32, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 32, 32, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 32, 32, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 32, 32, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 32, 32, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 23,718,790 (90.48 MB)

 Trainable params: 131,076 (512.02 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 2 (12.00 B)

In [14]:
# MobileNet Model
mobilenet_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in mobilenet_base_model.layers:
    layer.trainable = False

mobilenet_x = mobilenet_base_model.output
mobilenet_x = layers.Flatten()(mobilenet_x)  # Flatten the output
mobilenet_x = layers.Dropout(0.5)(mobilenet_x)
mobilenet_predictions = layers.Dense(4, activation='softmax')(mobilenet_x)

mobilenet_model = Model(inputs=mobilenet_base_model.input, outputs=mobilenet_predictions)

mobilenet_model.compile(optimizer=Adam(learning_rate=0.01),
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

checkpoint = ModelCheckpoint('mobilenet_best_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)

mobilenet_history = mobilenet_model.fit(aug_train_ds, validation_data=val_ds, epochs=15, callbacks=[checkpoint])

timestamp = time.strftime("%Y%m%d-%H%M%S")

# Make file name with timestamp
model_name = "mobilenet_model"
file_path = f"{model_name}_{timestamp}.h5"

# Save the model
mobilenet_model.save(file_path)
print(f'Model saved as {file_path}')

# Append description and history to .h5 file
description = "Final MobilenetV2 Model"
history_dict = {
    'loss': mobilenet_history.history['loss'],
    'accuracy': mobilenet_history.history['accuracy'],
    'val_loss': mobilenet_history.history['val_loss'],
    'val_accuracy': mobilenet_history.history['val_accuracy']
}
history_json = json.dumps(history_dict)

with h5py.File(file_path, 'a') as f:
    f.attrs['description'] = description
    f.attrs['training_history'] = history_json

print('Description and training history saved to the model file.')

# Load the description and training history
with h5py.File(file_path, 'r') as f:
    loaded_description = f.attrs['description']
    loaded_history_json = f.attrs['training_history']
    loaded_history = json.loads(loaded_history_json)

print(f'Loaded description: {loaded_description}')
print(f'Loaded history: {loaded_history}')

# Load the saved model
saved_model = load_model(file_path)
saved_model.summary()


Epoch 1/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 26s 67ms/step - accuracy: 0.4955 - loss: 12.0447 - val_accuracy: 0.4000 - val_loss: 15.8084
Epoch 2/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.6160 - loss: 11.6855 - val_accuracy: 0.3875 - val_loss: 28.6825
Epoch 3/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.6543 - loss: 11.0830 - val_accuracy: 0.5250 - val_loss: 17.4875
Epoch 4/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.6792 - loss: 10.4083 - val_accuracy: 0.4125 - val_loss: 30.9011
Epoch 5/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.6976 - loss: 10.9283 - val_accuracy: 0.4250 - val_loss: 31.5371
Epoch 6/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.6963 - loss: 11.8960 - val_accuracy: 0.4500 - val_loss: 45.1488
Epoch 7/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.7333 - loss: 10.6813 - val_accuracy: 0.5000 - val_loss: 24.2206
Epoch 8/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.7306 - l

Model saved as mobilenet_model_20241117-210652.h5
Description and training history saved to the model file.
Loaded description: Final MobilenetV2 Model
Loaded history: {'loss': [9.896991729736328, 10.885725021362305, 10.351579666137695, 9.803372383117676, 10.573202133178711, 11.111522674560547, 9.907712936401367, 10.146424293518066, 10.746733665466309, 9.59021282196045, 9.462979316711426, 9.84426498413086, 9.566265106201172, 10.21585464477539, 10.029574394226074], 'accuracy': [0.5526999831199646, 0.637499988079071, 0.6700999736785889, 0.6927000284194946, 0.7039999961853027, 0.7148000001907349, 0.739799976348877, 0.7407000064849854, 0.7354000210762024, 0.7666000127792358, 0.7620000243186951, 0.76419997215271, 0.7688000202178955, 0.7694000005722046, 0.777899980545044], 'val_loss': [15.808428764343262, 28.682476043701172, 17.487520217895508, 30.901098251342773, 31.537067413330078, 45.14883041381836, 24.22062873840332, 28.916412353515625, 31.7003116607666, 29.770360946655273, 32.7826919555

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)   │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 64, 64, 32)        │             864 │ input_layer_10[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 64, 64, 32)        │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 64, 64, 32)        │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 64, 64, 32)        │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 64, 64, 32)        │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 64, 64, 32)        │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 64, 64, 16)        │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 64, 64, 16)        │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 64, 64, 96)        │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 64, 64, 96)        │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 64, 64, 96)        │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 65, 65, 96)        │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 32, 32, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,339,910 (8.93 MB)

 Trainable params: 81,924 (320.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2 (12.00 B)

In [15]:
seed = seed


new_model = Sequential()

# Input layer
new_model.add(InputLayer(shape=(128, 128, 3)))


new_model.add(Conv2D(16, (3, 3), activation='relu', strides=(1, 1), padding='same', kernel_regularizer=l2(0.01)))
#new_model.add(BatchNormalization())
new_model.add(MaxPooling2D((2, 2)))
new_model.add(Dropout(0.1))

              
new_model.add(Conv2D(32, (3, 3), activation='relu', strides=(1, 1), padding='same', kernel_regularizer=l2(0.01)))
#new_model.add(BatchNormalization())
new_model.add(MaxPooling2D((2, 2)))
new_model.add(Dropout(0.2))

              
new_model.add(Conv2D(64, (3, 3), activation='relu', strides=(1, 1), padding='same', kernel_regularizer=l2(0.01)))  
#new_model.add(BatchNormalization())
new_model.add(MaxPooling2D((2, 2)))
new_model.add(Dropout(0.4))



new_model.add(Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same', kernel_regularizer=l2(0.01)))
#new_model.add(BatchNormalization())
new_model.add(MaxPooling2D((2, 2)))

new_model.add(Flatten())

# Output layers
new_model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
new_model.add(Dropout(0.5))
new_model.add(Dense(units=4, activation='softmax', name='output'))


############################################################################################################################
#Compile the model
new_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
############################################################################################################################
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00005)

checkpoint = ModelCheckpoint('new_best_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)

# Fit the model on data and choose # of epochs
new_model_history = new_model.fit(aug_train_ds, epochs=50, validation_data=val_ds, callbacks=[early_stopping, reduce_lr, checkpoint])
                            
############################################################################################################################




# Generate a timestamp
timestamp = time.strftime("%Y%m%d-%H%M%S")

# Make file name with timestamp
model_name = "EvenDistr"
file_path = f"{model_name}_{timestamp}.h5"

# Save the model
new_model.save(file_path)
print(f'Model saved as {file_path}')

# Append description and history to .h5 file
description = "100 images from each class, 10k aug samples"

history_dict = {
    'loss': new_model_history.history['loss'],
    'accuracy': new_model_history.history['accuracy'],
    'val_loss': new_model_history.history['val_loss'],
    'val_accuracy': new_model_history.history['val_accuracy']
}
history_json = json.dumps(history_dict)

with h5py.File(file_path, 'a') as f:
    f.attrs['description'] = description
    f.attrs['training_history'] = history_json

print('Description and training history saved to the model file.')

# Load the description and training history
with h5py.File(file_path, 'r') as f:
    loaded_description = f.attrs['description']
    loaded_history_json = f.attrs['training_history']
    loaded_history = json.loads(loaded_history_json)

print(f'Loaded description: {loaded_description}')
print(f'Loaded history: {loaded_history}')

# Load the saved model
saved_model = load_model(file_path)
saved_model.summary()



Epoch 1/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 17s 45ms/step - accuracy: 0.2696 - loss: 30.9366 - val_accuracy: 0.2625 - val_loss: 4.6325 - learning_rate: 0.0010
Epoch 2/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.2793 - loss: 4.4784 - val_accuracy: 0.3875 - val_loss: 4.0275 - learning_rate: 0.0010
Epoch 3/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.3156 - loss: 3.9111 - val_accuracy: 0.2875 - val_loss: 3.5541 - learning_rate: 0.0010
Epoch 4/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.3385 - loss: 3.4287 - val_accuracy: 0.4375 - val_loss: 3.1437 - learning_rate: 0.0010
Epoch 5/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - accuracy: 0.3755 - loss: 3.0607 - val_accuracy: 0.3750 - val_loss: 2.8089 - learning_rate: 0.0010
Epoch 6/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - accuracy: 0.4055 - loss: 2.7301 - val_accuracy: 0.3125 - val_loss: 2.5755 - learning_rate: 0.0010
Epoch 7/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - accuracy: 0.4399 - 

Model saved as EvenDistr_20241117-211437.h5
Description and training history saved to the model file.
Loaded description: 100 images from each class, 10k aug samples
Loaded history: {'loss': [10.310697555541992, 4.3229079246521, 3.793290615081787, 3.329258918762207, 2.979321002960205, 2.6638805866241455, 2.41641902923584, 2.1715450286865234, 1.9690136909484863, 1.8206119537353516, 1.670344352722168, 1.5737953186035156, 1.456054449081421, 1.377175211906433, 1.3095201253890991, 1.263609528541565, 1.1900334358215332, 1.1326682567596436, 1.0986082553863525, 0.8832477331161499, 0.783440887928009, 0.731680691242218, 0.6929981708526611, 0.6711151599884033, 0.63994961977005, 0.6136407256126404, 0.5917272567749023, 0.5385776162147522, 0.515065610408783], 'accuracy': [0.2655999958515167, 0.29440000653266907, 0.3118000030517578, 0.35510000586509705, 0.38040000200271606, 0.40880000591278076, 0.435699999332428, 0.4772000014781952, 0.49880000948905945, 0.5192000269889832, 0.5436000227928162, 0.55430

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 128)                 │       1,048,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,146,662 (4.37 MB)

 Trainable params: 1,146,660 (4.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)